# Gathered Notebook

 This notebook was generated by an experimental feature called "Gather". The intent is that it contains only the code and cells required to produce the same results as the cell originally selected for gathering. Please note that the Python analysis is quite conservative, so if it is unsure whether a line of code is necessary for execution, it will err on the side of including it.

**Please let us know if you are satisfied with what was gathered by [taking this survey](https://aka.ms/gathersurvey).**

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
titanic_df = pd.read_csv("titanic_data.csv")

In [ ]:
pandas_profiling.ProfileReport(titanic_df)

In [ ]:
age_median = titanic_df['Age'].median(skipna=True)
titanic_df['Age'].fillna(age_median, inplace=True)

In [ ]:
titanic_df['Embarked'].fillna("S", inplace=True)

In [ ]:
fare_median = titanic_df['Fare'].median(skipna=True)
titanic_df['Fare'].fillna(fare_median,inplace=True)

In [ ]:
titanic_df['TravelGroup']=titanic_df["SibSp"]+titanic_df["Parch"]
titanic_df['TravelAlone']=np.where(titanic_df['TravelGroup']>0, 0, 1) 

In [ ]:
titanic_df['TravelTotal'] = titanic_df['TravelGroup'] + 1

In [ ]:
le = LabelEncoder()
pclass_cat = le.fit_transform(titanic_df.Pclass)
sex_cat = le.fit_transform(titanic_df.Sex)
embarked_cat = le.fit_transform(titanic_df.Embarked)
titanic_df['pclass_cat'] = pclass_cat
titanic_df['sex_cat'] = sex_cat
titanic_df['embarked_cat'] = embarked_cat
features = ['pclass_cat', 'sex_cat', 'Age', 'Fare', 'embarked_cat', 'TravelAlone', 'TravelTotal']
data = titanic_df.reindex(features+['Survived'],axis=1)

In [ ]:
X = data.iloc[:, 0:7] 
Y = data.iloc[:, 7]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
gnb = GaussianNB().fit(X_train, y_train)
y_pred = gnb.predict(X_test)
NB_all_accuracy = accuracy_score(y_test,y_pred)

In [ ]:
tree = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 2, random_state=5).fit(X_train, y_train)
y_pred = tree.predict(X_test)
tree_all_accuracy = accuracy_score(y_test, y_pred)

In [ ]:
high_importance = list(zip(features, tree.feature_importances_))
high_importance = sorted(high_importance, key=lambda x: x[1], reverse=True)
high_importance = [tup[0] for tup in high_importance][:3]
X_train2 = X_train[high_importance]
X_test2 = X_test[high_importance]

In [ ]:
tree2 = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 2, random_state=5).fit(X_train2, y_train)
y_pred2 = tree2.predict(X_test2)
tree_imp_accuracy = accuracy_score(y_test, y_pred2)

In [ ]:
clf = RandomForestClassifier(n_jobs=2, random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
RF_all_accuracy = accuracy_score(y_test,y_pred)

In [ ]:
high_importance = list(zip(features, tree.feature_importances_))
high_importance = sorted(high_importance, key=lambda x: x[1], reverse=True)
high_importance = [tup[0] for tup in high_importance][:3]
X_train2 = X_train[high_importance]
X_test2 = X_test[high_importance]

In [ ]:
clf2 = RandomForestClassifier(n_jobs=2, random_state=0).fit(X_train2, y_train)
y_pred2 = clf2.predict(X_test2)
RF_imp_accuracy = accuracy_score(y_test,y_pred2)

In [ ]:
print('NB accuracy: {0:.2f}'.format(NB_all_accuracy))
print("Decision Tree:")
print('All attributes: {0:.2f}'.format(tree_all_accuracy))
print('High importance attributes: {0:.2f}'.format(tree_imp_accuracy))
print("Random Forsest:")
print('All attributes: {0:.2f}'.format(RF_all_accuracy))
print('High importance attributes: {0:.2f}'.format(RF_imp_accuracy))